In [5]:
from itertools import chain
from tqdm import tqdm
import multiprocessing
from pathlib import Path
from beat_this.inference import File2Beats

file2beats = File2Beats(device="cuda", dbn=True)
def process_songs(input_dir: Path, output_dir: Path, audio_dir: Path = Path('./audio')):
    output_dir.mkdir(exist_ok=True)
    # for input_midi_file in tqdm(chain(input_dir.glob('*.mid'),input_dir.glob('*.midi'))):
    #     output_midi_file = output_dir / input_midi_file.name
    #     process_song(input_midi_file, output_midi_file, audio_dir)

    n_shards = 10
    input_files = list(chain(input_dir.glob('*.mid'),input_dir.glob('*.midi')))
    input_files = [input_files[i::n_shards] for i in range(n_shards)]
    output_files = [output_dir / f'{i}.mid' for i in range(n_shards)]
    audio_dirs = [audio_dir for i in range(n_shards)]
    with multiprocessing.Pool(n_shards) as p:
        p.starmap(process_songs_shard, zip(input_files, output_files, audio_dirs))
        

In [6]:
if __name__ == '__main__':
    process_songs(Path('./midi'), Path('./processed_midi'))

  0%|          | 0/7 [00:00<?, ?it/s]